# Après Java 8

## Java 8

### Les lambdas expressions
Soit la liste d’instances de la classe Personne(email, prénom, nom, age) suivante : 

In [1]:
%%shell
gitpuller https://github.com/ebpro/sample-person.git develop /home/jovyan/work/src/samples/sample-person
cd "/home/jovyan/work/src/samples/sample-person"
mvn -quiet -ntp clean package

$ git fetch

$ git -c user.email=nbgitpuller@nbgitpuller.link -c user.name=nbgitpuller merge -Xours origin/develop

Already up to date.



In [2]:
List<String> addedJars = %jars "/home/jovyan/work/src/samples/sample-person/target/SamplePerson-*.jar";  
addedJars;

[/home/jovyan/work/src/samples/sample-person/target/SamplePerson-1.0-SNAPSHOT.jar]

In [3]:
import fr.univtln.bruno.samples.persons.Person;
List<Person> personnes = Arrays.asList(
                        Person.getInstance("pierre.durand@a.fr","Pierre", "Durand", 20), 
                        Person.getInstance("marie.durand@b.fr","Marie", "Durand", 14),
                        Person.getInstance("albert.martin@c.fr","Albert", "Martin", 30));
personnes;

[Person(email=pierre.durand@a.fr, firstName=Pierre, lastName=Durand, age=20), Person(email=marie.durand@b.fr, firstName=Marie, lastName=Durand, age=14), Person(email=albert.martin@c.fr, firstName=Albert, lastName=Martin, age=30)]

Si la classe `Person` redéfini `equals` et `hashcode` et implante l'interface `Comparable` en fonction de l'email on peut trier canoniquement la liste par email.

In [4]:
Collections.sort(personnes);
personnes;

[Person(email=albert.martin@c.fr, firstName=Albert, lastName=Martin, age=30), Person(email=marie.durand@b.fr, firstName=Marie, lastName=Durand, age=14), Person(email=pierre.durand@a.fr, firstName=Pierre, lastName=Durand, age=20)]

Si l’on souhaite trier la collection par âge, il faut définir une classe qui implante l’interface Comparator<Person> 
 

In [5]:
public class PersonComparatorByAge implements Comparator<Person> {
    public int compare(Person person1, Person person2) {
        return person1.getAge()-person2.getAge();
    }
}

Collections.sort(personnes, new PersonComparatorByAge());
personnes;

[Person(email=marie.durand@b.fr, firstName=Marie, lastName=Durand, age=14), Person(email=pierre.durand@a.fr, firstName=Pierre, lastName=Durand, age=20), Person(email=albert.martin@c.fr, firstName=Albert, lastName=Martin, age=30)]

Si le comparateur ne doit être utilisé qu'une fois on peut utiliser une classe anonyme.

In [6]:
//Tri par prénom, puis nom
Collections.sort(personnes, new Comparator<Person>() {
    @Override
    public int compare(Person o1, Person o2) {
        return (o1.getFirstName().equals(o2.getFirstName())?o1.getLastName().compareTo(o2.getLastName()):o1.getFirstName().compareTo(o2.getFirstName()));
    }
});
personnes;

[Person(email=albert.martin@c.fr, firstName=Albert, lastName=Martin, age=30), Person(email=marie.durand@b.fr, firstName=Marie, lastName=Durand, age=14), Person(email=pierre.durand@a.fr, firstName=Pierre, lastName=Durand, age=20)]

A partir de Java 8 on peut utiliser des lambda expressions. Elles peuvent être vues comme des instances de classes anonymes n'ayant qu'une seule méthode dont les types de retour et des paramètres sont inférés.

Une lambda est composée :
  * d'un ou ou de plusieurs noms de paramètres (alors entre parenthèses) éventuellement typés
  * du symbole →
  * d'un corps de fonction, s’il est réduit à une expression le return et les accolades sont facultatifs.


In [7]:
Collections.sort(personnes, (o1, o2) -> o1.getLastName().compareTo(o2.getLastName()));
personnes;

[Person(email=marie.durand@b.fr, firstName=Marie, lastName=Durand, age=14), Person(email=pierre.durand@a.fr, firstName=Pierre, lastName=Durand, age=20), Person(email=albert.martin@c.fr, firstName=Albert, lastName=Martin, age=30)]

## Les interfaces fonctionnelles

Java 8 définit le concept d’interfaces fonctionnelles (elles ont extactement une méthode). Elles permettent de manipuler des références vers des lambda expressions ou des méthodes. Une interface peut être définie comme fonctionnelle avec l’annotation @FunctionalInterface. Un ensemble d’interfaces fonctionnelles classiques est proposé dans le JDK :

  * `Function<T,R>` - takes an object of type T and returns R.
  * `Supplier<T>` - just returns an object of type T.
  * `Predicate<T>` - returns a boolean value based on input of type T.
  * `Consumer<T>` - performs an action with given object of type T.
  * `BiFunction<T,U>` - like Function but with two parameters.
  * `BiConsumer<T,U>` - like Consumer but with two parameters.
  * ... https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/util/function/package-summary.html
  
Nous allons voir des exemples avec quatre interfaces courament utilsées : `Function`, `Predicate`, `Supplier` et `Consumer`.

La `Function` applique un traitement sur son unique paramètre. 

Il est par exemple possible de définir une variable `at` qui contient une référence vers une "fonction" donc avec un seul paramètre ici de type String qui elle ajoute un `@` en préfixe à une chaîne de caractères (donc avec comme type de retour String). Le nom de l'unique méthode d'une `Function` est `apply`.

In [8]:
import java.util.function.Function;
Function<String, String> at = name -> "@" + name;

On peut alors définir des méthodes qui prennent en paramètre une instance d'une interface fonctionnelle. 

Ici une méthode qui parcourt la liste de personnes, applique la Function sur le prénom et l'affiche.

In [9]:
public class AfficheurDePersonnes {
    public static void appliqueEtAfficher(Function<String,String> fonction) {
        for (Person p : personnes) System.out.println(fonction.apply(p.getFirstName()));
    }
}    

AfficheurDePersonnes.appliqueEtAfficher(at);

@Marie
@Pierre
@Albert


On peut aussi directement utiliser une lambda de types compatibles (noter que les types de retour et du paramètre `s` sont inférés à partir des paramètres de l'interface fonctionnelle attendue). 

In [10]:
AfficheurDePersonnes.appliqueEtAfficher(s->s.toUpperCase());

MARIE
PIERRE
ALBERT


Les `Supplier`s sont des fonctions sans paramètre qui produise une valeur et possède donc un seul paramètre celui de l'objet retourné. C'est une sorte de factory. Ainsi `Supplier<String>` fournis des chaines de caratères, `Supplier<Integer>` des objets représentant des entiers, ... . Pour les primitifs, il existe des variantes dédiées de l'interface (donc sans paramètres) comme `IntSupplier`.

On peut ainsi par exemple, définir deux fonctions qui affichent 10 entierss à partir de suppliers (Object ou primitif).

In [11]:
import java.util.function.Supplier;
import java.util.function.IntSupplier;
import java.util.Random;

public class AfficheurDeNombre {
    private static int nb = 10;
    public static void afficher(Supplier<Integer> supplier) {
        for (int i=0;i<nb;i++) System.out.print(supplier.get()+(i<nb-1?" ":"\n"));        
    }
    public static void afficher(IntSupplier supplier) {
        for (int i=0;i<nb;i++) System.out.print(supplier.getAsInt()+(i<nb-1?",":"\n"));
    }
} 

et les utiliser avec des lamdas conforme à l'interface Supplier.

In [12]:
AfficheurDeNombre.afficher(()->new Random().nextInt(6)+1);
AfficheurDeNombre.afficher(()->(new Random().nextInt(50)+1)*2);

3,1,1,5,1,5,5,1,3,6
66,96,34,66,88,6,26,16,94,50


L'opérateur `::` permet de récupérer une référence vers un constructeur, une méthode statique ou d'instance existante que l'on pourra manipuler à travers une interface fonctionnelle.

On peut ainsi définir une méthode qui ajoute des Personnes à une liste qui est créée à partir d'un Supplier en paramètre.

In [13]:
public void testSupplier(Supplier<List<Person>> plFactory) {
  List <Person> pl = plFactory.get();
  pl.add(Person.getInstance("pierre.durand@a.fr","Pierre", "Durand", 20)); 
  pl.add(Person.getInstance("marie.durand@b.fr","Marie", "Durand", 14));
  System.out.println(pl.getClass()+ "->" +pl);
}

Supplier<List<Person>> personListFactory = ArrayList::new;

testSupplier(personListFactory);

testSupplier(LinkedList::new);

class java.util.ArrayList->[Person(email=pierre.durand@a.fr, firstName=Pierre, lastName=Durand, age=20), Person(email=marie.durand@b.fr, firstName=Marie, lastName=Durand, age=14)]
class java.util.LinkedList->[Person(email=pierre.durand@a.fr, firstName=Pierre, lastName=Durand, age=20), Person(email=marie.durand@b.fr, firstName=Marie, lastName=Durand, age=14)]


L'interface `Consumer` a un paramètre, le type d'objet consommé et aucun type de retour.
L'interface `Predicate` a un paramètre, le type d'objet sur lequel s'applique le prédicat et renvoie un booléen.

In [14]:
import java.util.function.Consumer;
import java.util.function.Predicate;
Consumer<String> println = System.out::println;

public class ListProcessor<T> {    
    //Une méthode qui applique successivement un traitement sur 10 objets de type T si il vérifie un prédicat.
    //La méthode process a trois paramètres : le supplier fournit les objets, le prédicat la condition et le Consummer l'action.
    public void process(Supplier<T> supplier, Predicate<T> predicate, Consumer<T> consumer) {
        for (int i=0;i<10;i++) {
                T s = supplier.get();
                if (predicate.test(s)) consumer.accept(s);
        }
    }
}    

//On peut par exemple générer aléatoirement des String et n'afficher que celles qui commencent par une lettre.
new ListProcessor<String>().process(
    //Le Supplier
    ()-> {
        byte[] array = new byte[8]; 
        new Random().nextBytes(array);
        return new String(array, Charset.forName("UTF-8"));}, 
    //Le Predicate
    s->Character.isLetter(s.charAt(0)),
    //Le Consumer
    println);


XW9+X{
T�#]�b5
ic�?���
Ǹ�%��%O


### Applications à la liste de personnes
On peut donc retrouver des personnes avec un filtre.

En utilisant une classe générique pour la recherche :

In [15]:
public class Processor<T> {
    public List<T> find(Iterable<T> iterable, Predicate<T> predicate) {
        List<T> list = new ArrayList<>();
        for (T t : iterable)
            if (predicate.test(t))
                list.add(t);
        return list;
    }
}
 
Processor<Person> personProcessor = new Processor<>();

On peut donc utiliser une classe anonyme pour le critère :

In [16]:
System.out.println(personProcessor.find(personnes,
                new Predicate<Person>() {
                    @Override
                    public boolean test(Person p) {
                        return p.getLastName().equals("Durand")
                                && p.getAge() >= 18
                                && p.getAge() <= 25;
                    }
                }));        

[Person(email=pierre.durand@a.fr, firstName=Pierre, lastName=Durand, age=20)]


ou mieux avec une lambda expression

In [17]:
System.out.println(personProcessor.find(personnes,
                p -> p.getLastName().equals("Durand")
                        && p.getAge() >= 10
                        && p.getAge() <= 15
));

[Person(email=marie.durand@b.fr, firstName=Marie, lastName=Durand, age=14)]


## Les Streams

Un stream permet de représenter une séquence d’objets qui peut supporter l’exécution parallèle. La construction de stream peut être “lazzy”.

Un stream peut être créé au dessus d’une collection

In [18]:
import java.util.stream.Stream;

personnes.stream(); //Returns a sequential Stream with the collection as its source.
personnes.parallelStream(); //Returns a possibly parallel Stream with the collection as its source.

java.util.stream.ReferencePipeline$Head@a604904

Un stream peut être parcourus avec un foreach qui permet d’appliquer une fonction sur chaque élément au fur et à mesure de leur production.

In [19]:
personnes.stream().forEach(System.out::println);

Person(email=marie.durand@b.fr, firstName=Marie, lastName=Durand, age=14)
Person(email=pierre.durand@a.fr, firstName=Pierre, lastName=Durand, age=20)
Person(email=albert.martin@c.fr, firstName=Albert, lastName=Martin, age=30)


D’autres classes produisent des Stream comme les fichiers

In [20]:
try (Stream stream = Files.lines(Paths.get("/etc/legal"))) {
            stream.forEach(System.out::println);
}


The programs included with the Ubuntu system are free software;
the exact distribution terms for each program are described in the
individual files in /usr/share/doc/*/copyright.

Ubuntu comes with ABSOLUTELY NO WARRANTY, to the extent permitted by
applicable law.



ou les flux d'entrée/sortie (I/O stream)

In [21]:
try (FileReader fileReader = new FileReader("/etc/legal");
  BufferedReader br = new BufferedReader(fileReader)) {
  br.lines().forEach(System.out::println);
}


The programs included with the Ubuntu system are free software;
the exact distribution terms for each program are described in the
individual files in /usr/share/doc/*/copyright.

Ubuntu comes with ABSOLUTELY NO WARRANTY, to the extent permitted by
applicable law.



Il existe des versions plus performantes pour les primitifs IntStream, DoubleStream, et LongStream.

Un stream peut aussi être filtré.

In [22]:
personnes.stream().filter(p -> p.getAge() > 18).forEach(System.out::println);

Person(email=pierre.durand@a.fr, firstName=Pierre, lastName=Durand, age=20)
Person(email=albert.martin@c.fr, firstName=Albert, lastName=Martin, age=30)


Et il permet d’utiliser le modèle map/reduce.

In [23]:
double ageMoyen = personnes.parallelStream().mapToInt(Person::getAge).average().getAsDouble();
System.out.println(ageMoyen);

21.333333333333332


## Les interfaces en général
A partir de Java 8, il est possible de définir une implantation par défaut dans une interface ainsi que des méthodes statiques.

In [24]:
public interface ActionClass<T> {

    //Depuis Java 9 les méthodes statiques sont autorisées
    static String getInterfaceDescription() {return "Une interface pour emprunter en général";}
    
    void action(T t);
    
    default void action(List<T> list) {
        for(T t:list) action(t);
    }
}

public class UpperCaser implements ActionClass<String> {
    public void action(String s) {System.out.println(s.toUpperCase());}
}

//La factory .of() est apparue dans Java9 pour créer des Collections non mutables.
new UpperCaser().action(List.of("abc","def"));

ABC
DEF


## Optional<T>
Pour traiter plus simplement les possibles tests des cas de valeurs nulles, Java8 introduit le type `Optional`, un conteneur qui fournit des méthodes utilitaires dédiées.
    
La factory `of` vérifie que la référence n'est pas nulle et provoque une exception à la création sinon. 
    
~~La méthode `get()` retourne la valeur.~~ Depuis Java 9, on préfére l'utilise de `orElseThrow()` qui émet une `NoSuchElementException` en l'abscence de valeur.

In [25]:
String str1 = "value1";
Optional<String> optional1 = Optional.of(str1);
optional1.orElseThrow();

value1

In [26]:
String str1Bis = null; // Provoque une Exception
Optional<String> optional1 = Optional.of(str1Bis);

EvalException: null

Il est possible de manipuler des références éventuellement nulles.
Dans ce cas `OrElse()` est une alternative à pour obtenir une valeur par défault en cas de valeur nulle au lieu d'un Exception.

In [27]:
String str2 = "value2";
Optional<String> optional2 = Optional.ofNullable(str2);
optional2.orElseThrow();

value2

In [28]:
String str3 = null;
//String str3 = "value3"; 
Optional<String> optional3 = Optional.ofNullable(str3);
optional3.orElseThrow();

EvalException: No value present

In [29]:
String str3Bis = null;
//String str3Bis = "value3"; 
Optional<String> optional3Bis = Optional.ofNullable(str3Bis);
optional3Bis.orElse("defaultValue3");

defaultValue3

`OrElseGet(Supplier s)` est une variante plus efficace si le calcul de la valeur par défaut est complexe car alors la fonction n'est évaluée qu'en cas de valeur nulle.

Dans l'exemple ci dessous en cas de valeur nulle, une entrée alétoire du tableau est retournée.

In [30]:
String str4 = null;
//String str4 = "value3"; 
String[] values={"a1","b2","c3"};

Optional<String> optional4 = Optional.ofNullable(str4);

optional4.orElseGet( () -> values[new Random().nextInt(values.length)] );

b2

## Java 9 
### Modularisation

Java 9 introduit la notion de module. Un module regroupe des classes, permet de définir des dépendances entre modules et défini une API publique. Le JDK devient modulaire et les programmes utilisateurs peuvent l'être cela permet d'executer des programmes en consommant moins de mémoire. Les classes internes du JDK ne sont plus accessibles depuis l'application pour améliorer la sécurité. 

Plus de détails : https://openjdk.java.net/projects/jigsaw/quick-start

## Process
L'accès aux informations des processus systèmes est amélioré.
Les examples suivants illustre l'accès au processus de la JVM et à tous ceux du système.

In [31]:
ProcessHandle processHandle = ProcessHandle.current();
ProcessHandle.Info processInfo = processHandle.info();

System.out.println("PID: " + processHandle.pid());
System.out.println("Arguments: " + processInfo.arguments());
System.out.println("Command: " + processInfo.command());
System.out.println("Instant: " + processInfo.startInstant());
System.out.println("Total CPU duration: " + processInfo.totalCpuDuration());
System.out.println("User: " + processInfo.user());

PID: 942
Arguments: Optional[[Ljava.lang.String;@3453a3bb]
Command: Optional[/home/jovyan/.sdkman/candidates/java/17.0.0-tem/bin/java]
Instant: Optional[2021-11-09T22:46:16.990Z]
Total CPU duration: Optional[PT24.85S]
User: Optional[jovyan]


In [32]:
import java.util.stream.Stream;

Stream<ProcessHandle> liveProcesses = ProcessHandle.allProcesses();

liveProcesses.filter(ProcessHandle::isAlive).limit(3)
        .forEach(ph -> {
            System.out.println("PID: " + ph.pid());
            System.out.println("  Start: " + ph.info().startInstant());
            System.out.println("  User: " + ph.info().user());
        });

PID: 1
  Start: Optional[2021-11-09T20:37:47.540Z]
  User: Optional[jovyan]
PID: 8
  Start: Optional[2021-11-09T20:37:47.870Z]
  User: Optional[jovyan]
PID: 895
  Start: Optional[2021-11-09T22:39:58.760Z]
  User: Optional[jovyan]


//TODO : ProcessBuilder (Since Java 7)

## Inférence du type des variables locales
Depuis Java 10, le type des variables locales avec initialisation peut être inféré à la compilation avec le mot clé `var`.

In [33]:
var i = 3;
var myMap = new HashMap<Integer,String>();

myMap.put(1,"A");
myMap.put(2,"B");
myMap;

{1=A, 2=B}

Depuis Java 11, c'est possible dans les lambdas même si le type des paramètre est déjà inféré.
L'un des avantages est de pouvoir associer des modificateurs sans avoir à spécifier le type.

In [34]:
public @interface MyAnnotation { }

import java.util.function.BiFunction;
BiFunction<String,String,String> f1 = (s1, s2) -> s1 + s2;

BiFunction<String,String,String> f2 = (@MyAnnotation var s1, @MyAnnotation var s2) -> s1 + s2;

## Instance Of
Depuis Java 12 en preview, l'opérateur `instance of` permet de faire implicement un cast en cas de succès.

In [35]:
Object obj = Person.getInstance("pierre.durand@a.fr","Pierre", "Durand", 20);

if (obj instanceof Person p) {
    System.out.println(p.getFirstName());
}

Pierre


### Text blocks
Depuis Java 13, il est possible d'utiliser des litéraux multilignes pour les String.

In [36]:
String person1Json="""
{
    "fistname" : "Pierre",
    "lastname" : "Durand",
    "description" : "lorem ipsum.............\
..................\
..................lorem ipsum"
}    
""";
    
person1Json;

{
    "fistname" : "Pierre",
    "lastname" : "Durand",
    "description" : "lorem ipsum.................................................lorem ipsum"
}


### Records
Les `record` (depuis Java 14/15) permettent de définir très simplement des classes non mutables avec constructeurs, accesseurs (sans le préfixe get) et toString().

In [37]:
public record Dog(int id, String name){ };

Dog d1 = new Dog(1, "Rex");
Dog d2 = new Dog(2, "Médor");

System.out.println(d1.name());

d2;
            

Rex


Dog[id=2, name=Médor]

In [38]:
public record User(int id, String firstname){ 
        public User {
            if(id < 0) { throw new IllegalArgumentException("Id has to be greater than 0.");
        }
    }
};

new User(-1,"Pierre");

EvalException: Id has to be greater than 0.

## Sealed and Hidden classes

```Java
public final class B extends A  { };
public non-sealed class C extends A  { };

public abstract sealed class A
    permits B, C {
}
```

### Switch
//TODO : still a preview

```Java
Dog d1 = new Dog(1, "Rex");
User u1= new User(1,"Pierre");

public String check(Object o) {
    switch (obj) {
                case User u -> "I'm a user "+u.firstname();
                case Dog d -> "I'm a dog "+u.name();
                case null -> "NULL !";
                default -> "A simple object.";
    };
}

check(d1);
```